# 03 — Demo: Transformer Tutor de Matemáticas y Física

Notebook de demostración del modelo Transformer **v3** (versión final) con tokenización **BPE subword**.

1. Carga el modelo TransformerV3 (BPE, vocab_size=4000, answer head) desde checkpoints
2. Generación autoregresiva con métricas **reales basadas en logits** (confianza, perplexity)
3. Diagnóstico de cross-attention (¿selectiva o colapsada?)
4. Interfaz interactiva Gradio con selector de dominio (math / physics)
5. Evolución del proyecto: v1 (char) → v2 (BPE) → v3 (answer head + diversity loss)
6. Discusión técnica honesta: qué puede y qué NO puede el modelo

In [1]:
# === Setup: GPU workaround para Blackwell + imports ===
import os, sys, subprocess

# XLA/GPU config ANTES de importar TF
os.environ.setdefault("XLA_FLAGS", "--xla_gpu_cuda_data_dir=/usr/local/cuda-12.8")
os.environ.setdefault("TF_XLA_FLAGS", "--tf_xla_auto_jit=2")

try:
    import gradio as gr
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "gradio==4.44.0", "-q"])
    import gradio as gr

import tensorflow as tf
import numpy as np
import json
import time
import re

# GPU memory growth + Blackwell cast patch
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    _original_cast = tf.cast
    def _blackwell_cast(x, dtype, name=None):
        if tf.executing_eagerly():
            with tf.device('/CPU:0'):
                return _original_cast(x, dtype, name=name)
        return _original_cast(x, dtype, name=name)
    tf.cast = _blackwell_cast

# Ruta del proyecto
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
for p in [project_root, os.path.dirname(project_root)]:
    if p not in sys.path:
        sys.path.insert(0, p)

tf.get_logger().setLevel('ERROR')

print(f"✅ TensorFlow {tf.__version__}")
print(f"✅ Gradio {gr.__version__}")
print(f"✅ GPU: {tf.config.list_physical_devices('GPU')}")
print(f"✅ Proyecto: {project_root}")

/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.6.3) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/home/melissa/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2026-02-14 21:06:30.184152: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-14 21:06:30.918025: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions:

✅ TensorFlow 2.20.0
✅ Gradio 4.44.0
✅ GPU: []
✅ Proyecto: /home/melissa/transformer_math_physics_tutor


W0000 00:00:1771121193.743381   21020 gpu_device.cc:2342] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


## 1. Cargar Modelo v3 (Versión Final) y Resultados de Evaluación

In [8]:
# === Cargar modelo v3 (TransformerV3 con Answer Head) ===
from transformer_math_physics_tutor.data.subword_tokenizer import SubwordTokenizer
from transformer_math_physics_tutor.models.config import TransformerConfig
from transformer_math_physics_tutor.models.transformer_v3 import TransformerV3

# --- Cargar configuración ---
checkpoint_dir = os.path.join(project_root, 'checkpoints', 'v3_easy')
config = TransformerConfig.load(os.path.join(checkpoint_dir, 'config.json'))

# --- Cargar tokenizer BPE ---
tokenizer = SubwordTokenizer(os.path.join(checkpoint_dir, 'sp_tokenizer.model'))
config.vocab_size = tokenizer.vocab_size

# --- Crear e inicializar modelo v3 ---
ANSWER_SCALE = 1000.0

with tf.device('/CPU:0'):
    model = TransformerV3(config, answer_scale=ANSWER_SCALE)
    dummy_enc = tf.zeros((1, config.max_encoder_len), dtype=tf.int32)
    dummy_dec = tf.zeros((1, config.max_decoder_len), dtype=tf.int32)
    _ = model((dummy_enc, dummy_dec), training=False)

    # Cargar pesos
    weights_path = os.path.join(checkpoint_dir, 'model_weights.weights.h5')
    model.load_weights(weights_path)

num_params = model.count_params()
device = "GPU" if gpus else "CPU"

# --- Compiled predict step para Blackwell GPU ---
max_enc = config.max_encoder_len
max_dec = config.max_decoder_len
pad_id = tokenizer.pad_token_id

@tf.function(input_signature=[
    tf.TensorSpec(shape=[1, max_enc], dtype=tf.int32),
    tf.TensorSpec(shape=[1, max_dec], dtype=tf.int32),
])
def predict_step(enc_input, dec_input):
    """Forward pass compilado — retorna solo logits (sin answer head)."""
    output = model((enc_input, dec_input), training=False)
    return output[0]  # logits (ignorar answer_pred)

# --- Historial de entrenamiento ---
history_raw = None
history = None  # Fase 3 (la que nos interesa para plots)
final_train_acc = final_val_acc = 0.0
try:
    with open(os.path.join(checkpoint_dir, 'training_history.json'), 'r') as f:
        history_raw = json.load(f)
    # El JSON tiene estructura anidada: phase1/phase2/phase3
    history = history_raw.get('phase3', history_raw)
    final_train_acc = history['train_accuracy'][-1]
    if history.get('val_accuracy'):
        final_val_acc = history['val_accuracy'][-1]
except Exception:
    pass

# --- Reporte de evaluación ---
eval_report = None
try:
    with open(os.path.join(checkpoint_dir, 'evaluation_report.json'), 'r') as f:
        eval_report = json.load(f)
except Exception:
    pass

# --- Config dict para display ---
with open(os.path.join(checkpoint_dir, 'config.json'), 'r') as f:
    config_dict = json.load(f)

print(f"✅ Modelo v3 cargado: {num_params:,} parámetros ({os.path.basename(weights_path)})")
print(f"✅ Dispositivo: {device}")
print(f"✅ Vocab size: {tokenizer.vocab_size} (BPE subword)")
print(f"✅ Encoder max: {config.max_encoder_len} | Decoder max: {config.max_decoder_len}")
print(f"✅ Accuracy — Train: {final_train_acc:.1%}, Val: {final_val_acc:.1%}")
if eval_report:
    em = eval_report.get('exact_match', {})
    ta = eval_report.get('token_accuracy', {})
    ah = eval_report.get('answer_head', {})
    attn = eval_report.get('cross_attention_entropy', {})
    print(f"✅ Token Acc (test): {ta.get('test_acc', 0):.1%}")
    print(f"✅ Exact Match: {em.get('textual_correct', 0)}/{em.get('textual_total', 0)} = {em.get('textual_pct', 0):.1f}%")
    print(f"✅ Numeric Match: {em.get('numeric_correct', 0)}/{em.get('numeric_total', 0)} = {em.get('numeric_pct', 0):.1f}%")
    print(f"✅ Answer Head MAE: {ah.get('mae', 0):.1f}")

    for layer, val in sorted(attn.items()):
        status = "⚠️ UNIFORME" if val > 0.95 else "✅ SELECTIVA" if val < 0.80 else "🟡 PARCIAL"
        print(f"   {layer}: {val:.3f} → {status}")

Configuración cargada desde /home/melissa/transformer_math_physics_tutor/checkpoints/v3_easy/config.json
SubwordTokenizer cargado desde /home/melissa/transformer_math_physics_tutor/checkpoints/v3_easy/sp_tokenizer.model (4000 tokens)


/home/melissa/.local/lib/python3.10/site-packages/keras/src/layers/layer.py:970: UserWarning: Layer 'xla_dropout_24' (of type XLADropout) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/home/melissa/.local/lib/python3.10/site-packages/keras/src/layers/layer.py:970: UserWarning: Layer 'xla_dropout_26' (of type XLADropout) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/home/melissa/.local/lib/python3.10/site-packages/keras/src/layers/layer.py:970: UserWarning: Layer 'xla_dropout_28' (of type XLADropout) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  w

✅ Modelo v3 cargado: 10,514,849 parámetros (model_weights.weights.h5)
✅ Dispositivo: CPU
✅ Vocab size: 4000 (BPE subword)
✅ Encoder max: 128 | Decoder max: 256
✅ Accuracy — Train: 64.9%, Val: 73.8%
✅ Token Acc (test): 69.9%
✅ Exact Match: 3/100 = 3.0%
✅ Numeric Match: 3/86 = 3.5%
✅ Answer Head MAE: 298.8
   decoder_layer1_block2: 0.742 → ✅ SELECTIVA
   decoder_layer2_block2: 0.540 → ✅ SELECTIVA
   decoder_layer3_block2: 0.523 → ✅ SELECTIVA
   decoder_layer4_block2: 0.673 → ✅ SELECTIVA


## 2. Función de Generación v3 (BPE + Answer Head + @tf.function)

La función `generate_with_metrics` usa decodificación **autoregresiva con predict_step compilado** y calcula métricas basadas en **logits reales**:

- **Confianza**: $\text{confidence} = \frac{1}{T}\sum_{t=1}^{T} \max_v P(v \mid v_{<t})$ — promedio de la probabilidad máxima por paso
- **Perplexity**: $\text{PPL} = \exp\!\left(-\frac{1}{T}\sum_{t=1}^{T} \log P(v_t \mid v_{<t})\right)$ — sobre la secuencia seleccionada
- **Answer Head**: Predicción numérica directa del MLP de regresión (sin generar tokens)

In [9]:
def _pad_to_length(tokens, max_len, pad_id):
    """Paddea o trunca una secuencia a max_len."""
    if len(tokens) > max_len:
        return tokens[:max_len - 1] + [tokens[-1]]
    return tokens + [pad_id] * (max_len - len(tokens))


def _detect_ngram_repeat(tokens, n=8):
    """Detecta si los últimos n tokens se repiten antes."""
    if len(tokens) < n * 2:
        return False
    last_ngram = tokens[-n:]
    for i in range(len(tokens) - n * 2, max(0, len(tokens) - n * 5) - 1, -1):
        if tokens[i:i + n] == last_ngram:
            return True
    return False


def generate_with_metrics(problem, max_length=256, temperature=0.3,
                          top_k=10, repetition_penalty=1.3):
    """
    Genera respuesta v3 con métricas reales + answer head prediction.

    Returns dict con: text, confidence, perplexity, inference_time,
                      num_tokens, answer_head_pred
    """
    if not problem or not problem.strip():
        return {
            'text': "⚠️ Escribe un problema de matemáticas o física.",
            'confidence': 0.0, 'perplexity': 0.0,
            'inference_time': 0.0, 'num_tokens': 0,
            'answer_head_pred': None,
        }

    start_time = time.time()

    # Tokenizar y paddear encoder input
    encoder_tokens = tokenizer.encode(problem, add_special_tokens=True)
    encoder_tokens = _pad_to_length(encoder_tokens, max_enc, pad_id)
    encoder_input = tf.constant([encoder_tokens], dtype=tf.int32)

    # Answer head prediction (single forward pass)
    with tf.device('/CPU:0'):
        dec_dummy = tf.constant(
            [[tokenizer.start_token_id] + [pad_id] * (max_dec - 1)],
            dtype=tf.int32
        )
        output_full = model((encoder_input, dec_dummy), training=False)
        answer_head_pred = float(output_full[1].numpy()[0]) * ANSWER_SCALE

    # Decodificación autoregresiva
    decoder_tokens = [tokenizer.start_token_id]
    generated_tokens = []
    token_probs = []

    for step in range(max_length):
        dec_padded = decoder_tokens + [pad_id] * (max_dec - len(decoder_tokens))
        decoder_input = tf.constant([dec_padded[:max_dec]], dtype=tf.int32)

        predictions = predict_step(encoder_input, decoder_input)

        pos = min(len(decoder_tokens) - 1, max_dec - 1)
        logits = predictions[0, pos, :].numpy()

        # Probabilidades ANTES de manipulación (para métricas puras)
        probs = np.exp(logits - np.max(logits))
        probs = probs / probs.sum()

        # Repetition penalty
        logits_np = logits.copy()
        if repetition_penalty > 1.0 and generated_tokens:
            for tok_id in set(generated_tokens):
                if logits_np[tok_id] > 0:
                    logits_np[tok_id] /= repetition_penalty
                else:
                    logits_np[tok_id] *= repetition_penalty

        # Selección de token con top-k sampling
        if temperature <= 0.0:
            predicted_id = int(np.argmax(logits_np))
        else:
            scaled = logits_np / temperature
            if top_k > 0:
                top_indices = np.argpartition(scaled, -top_k)[-top_k:]
                mask = np.full_like(scaled, -1e9)
                mask[top_indices] = scaled[top_indices]
                scaled = mask
            exp_scaled = np.exp(scaled - np.max(scaled))
            sampling_probs = exp_scaled / exp_scaled.sum()
            predicted_id = int(np.random.choice(len(sampling_probs), p=sampling_probs))

        selected_prob = float(probs[predicted_id])
        token_probs.append(selected_prob)

        if predicted_id == tokenizer.end_token_id:
            break

        generated_tokens.append(predicted_id)

        if len(generated_tokens) > 20 and _detect_ngram_repeat(generated_tokens, n=10):
            break

        decoder_tokens.append(predicted_id)
        if len(decoder_tokens) >= max_dec:
            break

    inference_time = time.time() - start_time

    # Decodificar
    generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()

    # Métricas
    if token_probs:
        confidence = float(np.mean(token_probs))
        nll = -np.mean([np.log(p + 1e-10) for p in token_probs])
        perplexity = float(np.exp(min(nll, 20)))
    else:
        confidence = 0.0
        perplexity = 0.0

    return {
        'text': generated_text,
        'confidence': confidence,
        'perplexity': perplexity,
        'inference_time': inference_time,
        'num_tokens': len(generated_tokens),
        'answer_head_pred': round(answer_head_pred, 1),
    }


# === Test rápido ===
print("=" * 60)
print("TEST RÁPIDO — Math")
print("=" * 60)
r = generate_with_metrics("Janet has 3 apples and buys 5 more. How many does she have?")
print(f"  Respuesta: {r['text'][:200]}")
print(f"  Answer Head: {r['answer_head_pred']}")
print(f"  Confianza: {r['confidence']:.2%} | PPL: {r['perplexity']:.2f} | {r['num_tokens']} tokens | {r['inference_time']*1000:.0f}ms")

print("\nTEST RÁPIDO — Physics")
r2 = generate_with_metrics("A car travels at 60 km/h for 3 hours. How far does it go?")
print(f"  Respuesta: {r2['text'][:200]}")
print(f"  Answer Head: {r2['answer_head_pred']}")
print(f"  Confianza: {r2['confidence']:.2%} | PPL: {r2['perplexity']:.2f} | {r2['num_tokens']} tokens | {r2['inference_time']*1000:.0f}ms")

TEST RÁPIDO — Math
  Respuesta: Step 1: First find the total number of additional objects she learns: 60 objects + 20 sheep = <<60+20=70>>70 items Step 2: Then divide the number of kids in the cat by the number of kids to find the t
  Answer Head: 21.0
  Confianza: 66.72% | PPL: 2.03 | 98 tokens | 3562ms

TEST RÁPIDO — Physics
  Respuesta: Step 1: Use the formula distance = speed × time. Step 2: distance = 60 × 3 = 180 km. Answer: 180 km
  Answer Head: 75.1
  Confianza: 90.67% | PPL: 1.16 | 39 tokens | 1254ms


In [10]:
# === Diagnóstico v3: problemas de ambos dominios ===
test_problems = [
    ("Janet has 3 apples and buys 5 more. How many apples does she have?", "math"),
    ("A store sells pencils for $2 each. Tom buys 7 pencils. How much does he spend?", "math"),
    ("A train travels 60 miles per hour for 3 hours. How far does it go?", "math"),
    ("Sarah has 24 cookies and wants to share them equally among 6 friends. How many cookies does each friend get?", "math"),
    ("John has $50. He buys a book for $12 and a pen for $3. How much money does he have left?", "math"),
    ("A car accelerates from rest at 3 m/s² for 5 seconds. What is its final velocity?", "physics"),
    ("A 10 kg box is pushed with a force of 50 N. What is its acceleration?", "physics"),
    ("How much heat is needed to raise the temperature of 2 kg of water by 30°C?", "physics"),
    ("A circuit has a voltage of 12V and a resistance of 4Ω. What is the current?", "physics"),
    ("An object is dropped from 80 m. How long does it take to hit the ground?", "physics"),
]

print("=" * 70)
print("DIAGNÓSTICO v3 — Math + Physics")
print("=" * 70)
for problem, domain in test_problems:
    r = generate_with_metrics(problem, temperature=0.0, top_k=0)
    if r['confidence'] >= 0.8:
        status = "🟢"
    elif r['confidence'] >= 0.5:
        status = "🟡"
    else:
        status = "🔴"
    # Extraer la respuesta
    ans_match = re.search(r'Answer:\s*(.+?)(?:\n|$)', r['text'], re.IGNORECASE)
    answer = ans_match.group(1).strip() if ans_match else "N/A"
    print(f"\n{status} [{domain.upper()}] {problem}")
    print(f"   → Answer: {answer}  (head pred: {r['answer_head_pred']})")
    print(f"   Confianza: {r['confidence']:.2%} | PPL: {r['perplexity']:.2f} | {r['num_tokens']} tokens")

DIAGNÓSTICO v3 — Math + Physics

🟡 [MATH] Janet has 3 apples and buys 5 more. How many apples does she have?
   → Answer: 4  (head pred: 24.5)
   Confianza: 70.78% | PPL: 1.68 | 78 tokens

🟡 [MATH] A store sells pencils for $2 each. Tom buys 7 pencils. How much does he spend?
   → Answer: 12  (head pred: 14.6)
   Confianza: 75.33% | PPL: 1.59 | 67 tokens

🟢 [MATH] A train travels 60 miles per hour for 3 hours. How far does it go?
   → Answer: 180 miles  (head pred: 59.9)
   Confianza: 88.16% | PPL: 1.22 | 39 tokens

🟡 [MATH] Sarah has 24 cookies and wants to share them equally among 6 friends. How many cookies does each friend get?
   → Answer: 80  (head pred: 28.1)
   Confianza: 78.88% | PPL: 1.44 | 47 tokens

🟡 [MATH] John has $50. He buys a book for $12 and a pen for $3. How much money does he have left?
   → Answer: 30  (head pred: 61.2)
   Confianza: 79.19% | PPL: 1.43 | 51 tokens

🟢 [PHYSICS] A car accelerates from rest at 3 m/s² for 5 seconds. What is its final velocity?
   → An

## 2.5 Diagnóstico Profundo v3 — ¿Qué aprendió el modelo?

Investigamos:
1. **¿El encoder influye en la salida?** — Si cambiamos el problema, ¿cambia la respuesta?
2. **¿El encoder produce embeddings distintos?** — Con BPE + answer head, ¿las representaciones son más ricas?
3. **¿Cross-attention funciona?** — ¿El decoder atiende al encoder? (v3 rompió el colapso!)
4. **¿Teacher forcing ayuda?** — Si le damos los primeros tokens correctos, ¿puede continuar?

In [11]:
print("=" * 70)
print("DIAGNÓSTICO 1: ¿El encoder influye en la salida? (v3)")
print("  (Mismo decoder start, problemas distintos, greedy)")
print("=" * 70)

for prob_text in [
    "What is 2 + 3?",
    "A farmer has 100 cows and sells 30. How many cows remain?",
    "A ball is thrown upward with a velocity of 20 m/s. What is the maximum height?",
]:
    r = generate_with_metrics(prob_text, temperature=0.0, top_k=0)
    ans = re.search(r'Answer:\s*(.+?)(?:\n|$)', r['text'], re.IGNORECASE)
    answer = ans.group(1).strip() if ans else "N/A"
    print(f"\n  Problema: {prob_text}")
    print(f"  → Answer: {answer}  (head pred: {r['answer_head_pred']})")
    print(f"  Confianza: {r['confidence']:.2%} | Tokens: {r['num_tokens']}")

print("\n" + "=" * 70)
print("DIAGNÓSTICO 2: ¿El encoder produce embeddings distintos? (v3)")
print("=" * 70)

problems_to_compare = [
    "What is 5 + 3?",
    "A car moves at 10 m/s for 5 seconds. What distance does it travel?",
    "XYZXYZXYZ",
]

enc_outputs = []
for prob in problems_to_compare:
    enc_tokens = tokenizer.encode(prob, add_special_tokens=True)
    enc_padded = _pad_to_length(enc_tokens, config.max_encoder_len, tokenizer.pad_token_id)
    with tf.device('/CPU:0'):
        enc_input = tf.constant([enc_padded], dtype=tf.int32)
        enc_padding_mask = model.create_padding_mask(enc_input)
        enc_out = model.encoder(enc_input, training=False, mask=enc_padding_mask)
    enc_outputs.append(enc_out.numpy())
    n_real = len(tokenizer.encode(prob, add_special_tokens=True))
    print(f"  '{prob[:50]}' ({n_real} subword tokens) → "
          f"mean={enc_out.numpy().mean():.4f}, std={enc_out.numpy().std():.4f}, "
          f"norm={np.linalg.norm(enc_out.numpy()):.2f}")

from numpy.linalg import norm
for i in range(len(enc_outputs)):
    for j in range(i+1, len(enc_outputs)):
        a, b = enc_outputs[i].flatten(), enc_outputs[j].flatten()
        cos_sim = np.dot(a, b) / (norm(a) * norm(b) + 1e-8)
        print(f"  Cosine sim [{i}] vs [{j}]: {cos_sim:.4f}")

print("\n" + "=" * 70)
print("DIAGNÓSTICO 3: ¿Cross-attention selectiva? (v3)")
print("  (v1/v2 tenían entropy ~1.000 = 100% uniforme)")
print("  (v3 entrenó con diversity loss para romper el colapso)")
print("=" * 70)

prob = "What is 5 + 3?"
enc_tokens = tokenizer.encode(prob, add_special_tokens=True)
enc_padded = _pad_to_length(enc_tokens, config.max_encoder_len, tokenizer.pad_token_id)
n_real = len(enc_tokens)
with tf.device('/CPU:0'):
    enc_input = tf.constant([enc_padded], dtype=tf.int32)
    dec_input = tf.constant([[tokenizer.start_token_id]], dtype=tf.int32)

    enc_padding_mask = model.create_padding_mask(enc_input)
    dec_padding_mask = model.create_padding_mask(enc_input)
    look_ahead_mask = model.create_look_ahead_mask(tf.shape(dec_input)[1])
    dec_target_padding_mask = model.create_padding_mask(dec_input)
    combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)

    enc_output = model.encoder(enc_input, training=False, mask=enc_padding_mask)
    dec_output, attn_weights = model.decoder(
        dec_input, enc_output, training=False,
        look_ahead_mask=combined_mask, padding_mask=dec_padding_mask
    )

for layer_name, weights in attn_weights.items():
    if 'block2' in layer_name:
        w = weights.numpy()[0]  # (heads, tar_len, inp_len)
        real_w = w[:, :, :n_real]
        pad_w = w[:, :, n_real:]
        total = real_w.sum() + pad_w.sum()
        print(f"  {layer_name}: tokens reales={real_w.sum()/total:.1%}, padding={pad_w.sum()/total:.1%}")
        avg = w.mean(axis=0)[0, :n_real]
        avg = avg / (avg.sum() + 1e-10)
        top_pos = avg.argsort()[-5:][::-1]
        tokens_str = [tokenizer.decode([enc_padded[p]], skip_special_tokens=False) for p in top_pos]
        entropy = -np.sum(avg * np.log(avg + 1e-10))
        max_entropy = np.log(n_real)
        print(f"    Entropy: {entropy:.3f} / {max_entropy:.3f} (max) = {entropy/max_entropy:.1%} → "
              f"{'⚠️ UNIFORME' if entropy/max_entropy > 0.95 else '✅ Selectiva'}")
        print(f"    Top-5: {list(zip(tokens_str, [f'{avg[p]:.4f}' for p in top_pos]))}")

print("\n" + "=" * 70)
print("DIAGNÓSTICO 4: Teacher forcing — ¿puede continuar? (v3)")
print("=" * 70)

prob = "What is 5 + 3?"
partial = "Step 1: 5 + 3 = "
enc_tokens = tokenizer.encode(prob, add_special_tokens=True)
enc_padded = _pad_to_length(enc_tokens, config.max_encoder_len, pad_id)
enc_input = tf.constant([enc_padded], dtype=tf.int32)

sol_tokens = [tokenizer.start_token_id] + tokenizer.encode(partial, add_special_tokens=False)
generated = list(sol_tokens)

for _ in range(80):
    if len(generated) >= max_dec:
        break
    dec_padded = generated + [pad_id] * (max_dec - len(generated))
    dec_input = tf.constant([dec_padded[:max_dec]], dtype=tf.int32)
    predictions = predict_step(enc_input, dec_input)
    pos = len(generated) - 1
    next_id = int(np.argmax(predictions[0, pos, :].numpy()))
    if next_id == tokenizer.end_token_id:
        break
    generated.append(next_id)

full_text = tokenizer.decode(generated[1:], skip_special_tokens=True)
print(f"  Problema: '{prob}'")
print(f"  Teacher forcing: '{partial}'")
print(f"  Continuación: '{full_text[:200]}'")

print("\n" + "=" * 70)
print("═══ RESUMEN DEL DIAGNÓSTICO v3 ═══")
print("=" * 70)
print("""
  RESULTADOS v3 (BPE subword + Answer Head + Diversity Loss):

  ✅ LOGROS PRINCIPALES:
  → Cross-attention SELECTIVA (entropy 0.52-0.74 vs 1.000 en v1/v2)
  → Genera soluciones con formato Step/Answer correcto
  → Fórmulas de física correctas (F=ma, V=IR, KE=½mv², v=fλ, d=v×t)
  → Answer head predice valores numéricos razonables
  → Token accuracy ~70% (subword tokens significativos)
  → Exact match 3.0% (textual) / 3.5% (numérico)

  ⚠️ LIMITACIONES:
  → Números específicos casi siempre incorrectos en problemas de math
  → El modelo genera soluciones memoryzadas cuando no reconoce el patrón
  → 5,729 ejemplos de entrenamiento insuficientes para generalización numérica
  → Se necesitaría un mecanismo de copia (copy mechanism) para copiar 
    números del input al output

  ¿POR QUÉ se rompió la cross-attention en v3? (vs colapsada en v1/v2)
  → Entrenamiento en 3 fases:
    1. Fase 1: Encoder pre-entrenado con answer head (gradiente directo)
    2. Fase 2: Cross-attention reinicializada + diversity loss alta
    3. Fase 3: Fine-tuning completo con learning rate baja
""")
print("=" * 70)

DIAGNÓSTICO 1: ¿El encoder influye en la salida? (v3)
  (Mismo decoder start, problemas distintos, greedy)

  Problema: What is 2 + 3?
  → Answer: N/A  (head pred: 287.6)
  Confianza: 57.86% | Tokens: 38

  Problema: A farmer has 100 cows and sells 30. How many cows remain?
  → Answer: 60  (head pred: 366.1)
  Confianza: 74.10% | Tokens: 60

  Problema: A ball is thrown upward with a velocity of 20 m/s. What is the maximum height?
  → Answer: 64 m/s  (head pred: 71.3)
  Confianza: 88.47% | Tokens: 58

DIAGNÓSTICO 2: ¿El encoder produce embeddings distintos? (v3)
  'What is 5 + 3?' (10 subword tokens) → mean=-0.0006, std=0.0866, norm=15.68
  'A car moves at 10 m/s for 5 seconds. What distance' (23 subword tokens) → mean=-0.0011, std=0.1141, norm=20.66
  'XYZXYZXYZ' (11 subword tokens) → mean=-0.0001, std=0.0813, norm=14.72
  Cosine sim [0] vs [1]: 0.3352
  Cosine sim [0] vs [2]: 0.3402
  Cosine sim [1] vs [2]: 0.4680

DIAGNÓSTICO 3: ¿Cross-attention selectiva? (v3)
  (v1/v2 tenían entro

## 3. Interfaz Gradio — Demo Interactiva

Layout:
- **Columna izquierda**: Entrada del problema, selector de dominio, botón de resolver, solución generada, respuesta destacada
- **Columna derecha**: Métricas de generación, predicción del answer head, detalles de arquitectura

In [12]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import tempfile


def create_training_plot():
    """Crea gráfica de entrenamiento y la guarda como archivo temporal."""
    if not history:
        return None
    fig, axes = plt.subplots(1, 3, figsize=(14, 3.5))

    # Loss
    axes[0].plot(history['train_loss'], label='Train', linewidth=2, color='#4f46e5')
    if history.get('val_loss'):
        axes[0].plot(history['val_loss'], label='Val', linewidth=2, color='#f97316')
    axes[0].set_title('Loss', fontsize=13, fontweight='bold')
    axes[0].set_xlabel('Época')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

    # Accuracy
    axes[1].plot(history['train_accuracy'], label='Train', linewidth=2, color='#4f46e5')
    if history.get('val_accuracy'):
        axes[1].plot(history['val_accuracy'], label='Val', linewidth=2, color='#f97316')
    axes[1].set_title('Token Accuracy (subword)', fontsize=13, fontweight='bold')
    axes[1].set_xlabel('Época')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)

    # Answer MAE
    if history.get('train_answer_mae'):
        axes[2].plot(history['train_answer_mae'], label='Train MAE', linewidth=2, color='#10b981')
        axes[2].set_title('Answer Head MAE', fontsize=13, fontweight='bold')
        axes[2].set_xlabel('Época')
        axes[2].legend()
        axes[2].grid(True, alpha=0.3)

    plt.tight_layout()
    tmp = tempfile.NamedTemporaryFile(suffix='.png', delete=False)
    plt.savefig(tmp.name, format='png', dpi=100, bbox_inches='tight')
    plt.close(fig)
    return tmp.name


def extract_answer_highlight(text):
    """Extrae la línea Answer: y la resalta."""
    match = re.search(r"(Answer:\s*.+?)(?:\n|$)", text, re.IGNORECASE)
    if match:
        return match.group(1).strip()
    return "No se detectó línea Answer:"


def tutor_interface(problem, domain):
    """Wrapper para Gradio: genera respuesta con métricas."""
    result = generate_with_metrics(problem, max_length=config.max_decoder_len)

    if result['confidence'] >= 0.8:
        conf_indicator = "🟢 Alta"
    elif result['confidence'] >= 0.5:
        conf_indicator = "🟡 Media"
    else:
        conf_indicator = "🔴 Baja"

    answer_line = extract_answer_highlight(result['text'])
    head_info = f"🧠 Answer Head ≈ {result['answer_head_pred']}" if result.get('answer_head_pred') is not None else ""
    if head_info:
        answer_line += f"\n\n{head_info}"

    return (
        result['text'],
        f"**{answer_line}**",
        result['confidence'],
        conf_indicator,
        f"{result['perplexity']:.2f}",
        f"{result['inference_time']*1000:.0f}",
        result['num_tokens']
    )


# === Dataset stats ===
dataset_stats = ""
try:
    with open(os.path.join(project_root, 'data', 'combined_easy.json'), 'r') as f:
        all_data = json.load(f)
    n_math = sum(1 for d in all_data if d.get('domain') == 'math')
    n_phys = sum(1 for d in all_data if d.get('domain') == 'physics')
    n_train = sum(1 for d in all_data if d.get('split') == 'train')
    dataset_stats = f"Dataset: **{len(all_data):,}** problemas ({n_math:,} math + {n_phys:,} physics) | Train: **{n_train:,}**"
except Exception:
    dataset_stats = "Dataset: combined_easy.json"

# === Evaluation summary ===
eval_summary = ""
if eval_report:
    ta = eval_report.get('token_accuracy', {})
    em = eval_report.get('exact_match', {})
    ah = eval_report.get('answer_head', {})
    eval_summary = (f"Token Acc (test): **{ta.get('test_acc', 0):.1%}** | "
                    f"Exact Match: **{em.get('textual_pct', 0):.1f}%** | "
                    f"Numeric: **{em.get('numeric_pct', 0):.1f}%** | "
                    f"Head MAE: **{ah.get('mae', 0):.1f}**")


# ═══════════════════════════════════════════════════════════
#  INTERFAZ GRADIO BLOCKS — v3 (Modelo Final)
# ═══════════════════════════════════════════════════════════
theme = gr.themes.Soft(primary_hue="indigo", secondary_hue="blue")

with gr.Blocks(theme=theme, title="Transformer Tutor — Math & Physics") as demo:

    gr.Markdown(f"""
    # 🎓 Transformer Tutor — Matemáticas y Física

    Modelo **Encoder-Decoder Transformer v3** desde cero |
    **{num_params:,}** parámetros | Tokenización: **BPE ({tokenizer.vocab_size} tokens)** |
    Dispositivo: **{device}** |
    Train: **{final_train_acc:.1%}** | Val: **{final_val_acc:.1%}** |
    {eval_summary}

    {dataset_stats}
    """)

    with gr.Row():

        # ═══ COLUMNA IZQUIERDA: Entrada / Salida ═══
        with gr.Column(scale=3):

            with gr.Row():
                input_box = gr.Textbox(
                    label="📝 Problema",
                    placeholder="Ej: A car travels at 60 km/h for 3 hours. How far does it go?",
                    lines=3,
                    scale=4
                )
                domain_selector = gr.Radio(
                    choices=["math", "physics"],
                    value="math",
                    label="🔬 Dominio",
                    scale=1
                )

            with gr.Row():
                solve_btn = gr.Button("🤔 Resolver", variant="primary", scale=3)
                clear_btn = gr.Button("🗑️ Limpiar", variant="secondary", scale=1)

            output_box = gr.Textbox(
                label="🤖 Solución (paso a paso)",
                lines=8,
                show_copy_button=True
            )

            answer_box = gr.Markdown(
                label="Respuesta",
                value="*Esperando problema...*"
            )

            gr.Markdown("### 📚 Ejemplos")
            with gr.Tab("Matemáticas"):
                gr.Examples(
                    examples=[
                        ["Janet has 3 apples and buys 5 more. How many apples does she have?", "math"],
                        ["A store sells pencils for $2 each. Tom buys 7 pencils. How much does he spend?", "math"],
                        ["Sarah has 24 cookies and shares equally among 6 friends. How many does each get?", "math"],
                        ["A farmer has 15 chickens. Each lays 2 eggs per day. How many eggs in 3 days?", "math"],
                        ["John has $50. He buys a book for $12 and a pen for $3. How much is left?", "math"],
                    ],
                    inputs=[input_box, domain_selector],
                    label=""
                )
            with gr.Tab("Física"):
                gr.Examples(
                    examples=[
                        ["A car travels at 60 km/h for 3 hours. How far does it go?", "physics"],
                        ["A 10 kg box is pushed with a force of 50 N. What is its acceleration?", "physics"],
                        ["A wave has frequency 100 Hz and wavelength 0.1 m. What is the wave speed?", "physics"],
                        ["A circuit has voltage 12V and resistance 4Ω. What is the current?", "physics"],
                        ["What is the kinetic energy of a 4 kg object moving at 5 m/s?", "physics"],
                    ],
                    inputs=[input_box, domain_selector],
                    label=""
                )

        # ═══ COLUMNA DERECHA: Métricas ═══
        with gr.Column(scale=2):

            gr.Markdown("### 📊 Métricas de Generación")
            with gr.Group():
                conf_slider = gr.Slider(0, 1, value=0, label="Confianza", interactive=False)
                conf_text = gr.Textbox(label="Estado", value="⏳ Esperando...", interactive=False)
                perp_text = gr.Textbox(label="Perplexity", value="--", interactive=False)
                time_text = gr.Textbox(label="Tiempo (ms)", value="--", interactive=False)
                tokens_num = gr.Number(label="Tokens generados", value=0)

            gr.Markdown("### 🧠 Arquitectura v3")
            with gr.Row():
                gr.Textbox(str(config_dict['num_layers']), label="Capas", interactive=False)
                gr.Textbox(str(config_dict['d_model']), label="d_model", interactive=False)
            with gr.Row():
                gr.Textbox(str(config_dict['num_heads']), label="Heads", interactive=False)
                gr.Textbox(str(config_dict.get('dff', 1024)), label="dff", interactive=False)
            with gr.Row():
                gr.Textbox(str(config_dict['dropout_rate']), label="Dropout", interactive=False)
                gr.Textbox(str(tokenizer.vocab_size), label="Vocab (BPE)", interactive=False)

    # ── Acordeón: Detalles Técnicos ───────────────────────
    with gr.Accordion("🔍 Detalles Técnicos", open=False):
        gr.Markdown(f"""
**Componentes implementados desde cero (sin usar capas pre-hechas):**
- Scaled Dot-Product Attention: `softmax(QK^T / √d_k) V`
- Multi-Head Attention ({config_dict['num_heads']} cabezas, depth={config_dict['d_model'] // config_dict['num_heads']})
- Positional Encoding sinusoidal
- Encoder: Self-Attn → Add&Norm → FFN → Add&Norm (×{config_dict['num_layers']})
- Decoder: Masked Self-Attn → Cross-Attn → FFN (×{config_dict['num_layers']})
- Answer Regression Head: MLP que predice el valor numérico de la respuesta

**Dataset (easy subset):**
- GSM8K: problemas de aritmética con soluciones paso a paso
- Física (templates): problemas de cinemática, dinámica, termodinámica, ondas, electricidad
- Total: ~6,881 problemas fáciles (train: ~5,729)

**Tokenización:** SentencePiece BPE ({tokenizer.vocab_size} tokens, split_digits=True, byte_fallback=True)
- Encoder max length: {config.max_encoder_len} subword tokens
- Decoder max length: {config.max_decoder_len} subword tokens

**Entrenamiento v3 (3 fases):**
1. **Fase 1 — Encoder pre-training**: Decoder congelado, solo encoder + answer head (30 epochs)
2. **Fase 2 — Cross-attention**: Encoder congelado, cross-attention reinicializada, diversity loss alta (100 epochs)
3. **Fase 3 — Fine-tuning**: Todo descongelado con learning rate baja (50 epochs)
- Optimizador: Adam (β₁=0.9, β₂=0.98) con warmup + inverse sqrt decay
- Loss combinada: seq2seq + λ·answer_loss + μ·diversity_loss
- GPU: NVIDIA RTX 5060 (Blackwell) con workarounds XLA/CUDA

**Inferencia:**
- Forward pass compilado con @tf.function + XLA (Blackwell compatible)
- Decodificación: Top-k sampling (k=10, temperature=0.3)
- Repetition penalty: 1.3
        """)

    # ── Acordeón: Curvas de Entrenamiento ─────────────────
    if history:
        with gr.Accordion("📈 Curvas de Entrenamiento", open=False):
            plot_img = create_training_plot()
            if plot_img:
                gr.Image(value=plot_img, label="Historia de entrenamiento v3")
            gr.Markdown(f"""
**Resumen del entrenamiento v3 (Fase 3 — fine-tuning):**
- Épocas completadas: {len(history.get('train_loss', []))}
- Loss final — Train: {history.get('train_loss', [0])[-1]:.4f} | Val: {history.get('val_loss', [0])[-1]:.4f}
- Accuracy final — Train: {history.get('train_accuracy', [0])[-1]:.4f} | Val: {history.get('val_accuracy', [0])[-1]:.4f}
- Answer MAE final: {history.get('train_answer_mae', [0])[-1]:.1f}
            """)

    # ── Acordeón: Evaluación ──────────────────────────────
    with gr.Accordion("📋 Resultados de Evaluación", open=False):
        if eval_report:
            ta = eval_report.get('token_accuracy', {})
            em = eval_report.get('exact_match', {})
            ah = eval_report.get('answer_head', {})
            at = eval_report.get('cross_attention_entropy', {})
            eval_md = f"""
**Token-level Accuracy** (predicción del siguiente subword token):
- Validación: {ta.get('val_acc', 0):.1%}
- Test: {ta.get('test_acc', 0):.1%}

**Exact Match (línea Answer:)** sobre {em.get('textual_total', 0)} problemas de test:
- Textual: **{em.get('textual_correct', 0)}/{em.get('textual_total', 0)} = {em.get('textual_pct', 0):.1f}%**
- Numérico (±0.5): **{em.get('numeric_correct', 0)}/{em.get('numeric_total', 0)} = {em.get('numeric_pct', 0):.1f}%**

**Answer Head Regression:**
- MAE: {ah.get('mae', 0):.1f}
- Exactos (±0.5): {ah.get('exact', 0)}/{ah.get('total', 0)} = {ah.get('exact_pct', 0):.1f}%

**Cross-Attention Entropy** (0.0 = perfectly focused, 1.0 = uniform/collapsed):
"""
            for layer, val in sorted(at.items()):
                status = "⚠️ UNIFORME" if val > 0.95 else "✅ SELECTIVA" if val < 0.80 else "🟡 PARCIAL"
                eval_md += f"- {layer}: **{val:.3f}** → {status}\n"

            eval_md += "\n**Ejemplos de predicciones:**\n"
            for ex in eval_report.get('examples', [])[:8]:
                icon = "✅" if ex.get('correct', False) else "❌"
                eval_md += f"- {icon} [{ex.get('domain','')}] \"{ex.get('problem','')[:60]}...\" → Esperado: `{ex.get('ref_answer','')}` | Predicho: `{ex.get('pred_answer','')}`\n"

            gr.Markdown(eval_md)
        else:
            gr.Markdown("*No se encontró `evaluation_report.json`.*")

    # ── Acordeón: Evolución v1 → v2 → v3 ─────────────────
    with gr.Accordion("🔄 Evolución v1 → v2 → v3", open=False):
        gr.Markdown("""
### Evolución del modelo

| Aspecto | v1 (char-level) | v2 (BPE subword) | v3 (answer head + diversity) |
|---------|----------------|-----------------|------------------------------|
| **Tokenización** | Character (135 tokens) | BPE (4,000 tokens) | BPE (4,000 tokens) |
| **Parámetros** | ~5.5M | ~7.5M | ~10.5M |
| **Token Accuracy** | ~82% (chars) | ~70% (subwords) | ~70% (subwords) |
| **Exact Match** | 0% | ~2% | **~3.5%** |
| **Cross-attention** | ⚠️ Colapsada (1.000) | ⚠️ Colapsada (1.000) | **✅ Selectiva (0.52-0.74)** |
| **Fórmulas física** | ❌ No | 🟡 A veces | **✅ Consistentes** |
| **Answer Head** | ❌ No | ❌ No | **✅ MAE ~299** |

### ¿Qué cambió en cada versión?
- **v1 → v2**: Tokenización BPE (SentencePiece) en lugar de character-level
- **v2 → v3**: Answer regression head + diversity loss + entrenamiento en 3 fases

### Logro principal de v3:
**Romper el colapso de cross-attention** (entropy 1.000 → 0.52-0.74). En v1 y v2, el decoder
ignoraba completamente el encoder. En v3, el decoder atiende selectivamente a tokens del input.
        """)

    # ── Acordeón: Limitaciones ────────────────────────────
    with gr.Accordion("⚠️ Limitaciones y Análisis Honesto", open=True):
        gr.Markdown("""
### ¿Qué puede y qué NO puede este modelo?

**✅ Lo que el modelo SÍ demuestra:**
1. **Arquitectura Transformer correcta**: Encoder-Decoder completo implementado desde cero
2. **Tokenización BPE funcional**: SentencePiece con 4,000 tokens, split_digits, byte_fallback
3. **Formato aprendido**: Genera consistentemente "Step 1: ... <<calc>> ... Answer: ..."
4. **Fórmulas de física**: Identifica y aplica F=ma, V=IR, KE=½mv², v=fλ, d=v×t correctamente
5. **Cross-attention selectiva**: Primer modelo que realmente atiende al input (entropy 0.52-0.74)
6. **Answer Head**: Predicción numérica directa sin generar tokens
7. **Pipeline completo de ML**: Datos → tokenización → 3 fases de entrenamiento → evaluación → demo

**❌ Lo que el modelo NO puede hacer:**
1. **Cálculos numéricos correctos**: Los números específicos son casi siempre incorrectos (~3.5% exact match)
2. **Generalización a problemas nuevos**: Tiende a generar soluciones memorizadas
3. **Copiar números del input**: No tiene mecanismo de copia (copy mechanism / pointer network)

### ¿Por qué los números siguen incorrectos?

- **5,729 ejemplos** de entrenamiento son insuficientes para ~10.5M parámetros
- Sin **copy mechanism**, el modelo debe "re-generar" cada número del input token por token
- Los Transformers pequeños son buenos para aprender **patrones estructurales** (fórmulas, formato)
  pero no para **cálculo numérico exacto**
- Modelos como GPT-4 usan 175B+ parámetros y trillones de tokens de entrenamiento

### Valor pedagógico

Este proyecto demuestra la implementación completa de un Transformer from-scratch,
incluyendo técnicas avanzadas como:
- Entrenamiento multi-fase con freezing/unfreezing selectivo
- Reinicialización de cross-attention para romper simetría
- Diversity loss para prevenir colapso de atención
- Compatibilidad con GPU Blackwell (workarounds XLA/CUDA)
- Evaluación honesta con métricas apropiadas
        """)

    # ── Eventos ───────────────────────────────────────────
    solve_btn.click(
        fn=tutor_interface,
        inputs=[input_box, domain_selector],
        outputs=[output_box, answer_box, conf_slider, conf_text, perp_text, time_text, tokens_num]
    )

    clear_btn.click(
        fn=lambda: ("", "math", "", "*Esperando problema...*", 0, "⏳ Esperando...", "--", "--", 0),
        inputs=None,
        outputs=[input_box, domain_selector, output_box, answer_box, conf_slider, conf_text, perp_text, time_text, tokens_num]
    )

# Lanzar
demo.launch(inline=True, share=False)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


/home/melissa/.local/lib/python3.10/site-packages/gradio/analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.44.0, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(
/home/melissa/.local/lib/python3.10/site-packages/keras/src/layers/layer.py:970: UserWarning: Layer 'xla_dropout_24' (of type XLADropout) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/home/melissa/.local/lib/python3.10/site-packages/keras/src/layers/layer.py:970: UserWarning: Layer 'xla_dropout_26' (of type XLADropout) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/home/melissa/.local/lib/python3.10/site-packages/keras/src/layers/layer.py:970: UserWarning: Layer 'xla_dro